In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
import pandas as pd

In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  
df.head(15)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [6]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [7]:
df1 = df.reset_index()

In [8]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.head(15)

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [9]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df3.head(15)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
print ('df Shape = ',df.shape)
print ('df1 Shape = ',df1.shape)
print ('df2 Shape = ',df2.shape)
print ('df3 Shape = ',df3.shape)

df Shape =  (103, 3)
df1 Shape =  (103, 4)
df2 Shape =  (103, 2)
df3 Shape =  (103, 3)


QUESTION 2

In [11]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import geopy

In [13]:
df1

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,166,M4Y,Downtown Toronto,Church and Wellesley
100,169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [14]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='duarfel@gmail.com')
df_geopy1 = df1
df_geopye = df1['Neighbourhood'].str.split(',', expand = True)
df_geopy1['address'] = 'Toronto, '+df_geopye[[0]]
df_geopy1.replace({'Neighbourhood':'Ontario Provincial Government'}, {'Neighbourhood':''}, inplace=True, regex=True)
df_geopy1.tail(15)

,index,PostalCode,Borough,Neighbourhood,address
88,143,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores","Toronto, New Toronto"
89,144,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...","Toronto, South Steeles"
90,145,M1W,Scarborough,"Steeles West, L'Amoreaux West","Toronto, Steeles West"
91,148,M4W,Downtown Toronto,Rosedale,"Toronto, Rosedale"
92,149,M5W,Downtown Toronto,Stn A PO Boxes,"Toronto, Stn A PO Boxes"
93,152,M8W,Etobicoke,"Alderwood, Long Branch","Toronto, Alderwood"
94,153,M9W,Etobicoke,"Northwest, West Humber - Clairville","Toronto, Northwest"
95,154,M1X,Scarborough,Upper Rouge,"Toronto, Upper Rouge"
96,157,M4X,Downtown Toronto,"St. James Town, Cabbagetown","Toronto, St. James Town"
97,158,M5X,Downtown Toronto,"First Canadian Place, Underground city","Toronto, First Canadian Place"


In [15]:
df_adr=df_geopy1[['address']]
df_adr['address'][11]

'Toronto, West Deane Park'

In [16]:
location = []

for x in range(0,len(df_adr)):
    location.append(
        {
            'location': geolocator.geocode(df_adr['address'][x])
        }
    )
    
df_locat=pd.DataFrame(location)
df_locat.shape

(103, 1)

In [17]:
loct= geolocator.geocode('Toronto')
latitude=loct.latitude
longitude=loct.longitude
loct

Location(Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada, (43.6534817, -79.3839347, 0.0))

In [18]:
locationLL = []
for x in range(0,len(df_locat)):
    if pd.isna(df_locat['location'][x])==0:
        
        locationLL.append(
            {
            
                'Latitude': df_locat['location'][x].latitude,
                'Longitude': df_locat['location'][x].longitude
            }
            
              )
    else:
        
            locationLL.append(
            
            {
            
                'Latitude': 43.6534817,
                'Longitude': -79.3839347
            }
    
    )
df_location=pd.DataFrame(locationLL)

In [19]:
df1['Latitude']= df_location['Latitude']
df1['Longitude']= df_location['Longitude']
df1.tail(15)

,index,PostalCode,Borough,Neighbourhood,address,Latitude,Longitude
88,143,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores","Toronto, New Toronto",43.600763,-79.505264
89,144,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...","Toronto, South Steeles",43.816178,-79.314538
90,145,M1W,Scarborough,"Steeles West, L'Amoreaux West","Toronto, Steeles West",43.816178,-79.314538
91,148,M4W,Downtown Toronto,Rosedale,"Toronto, Rosedale",43.678356,-79.380746
92,149,M5W,Downtown Toronto,Stn A PO Boxes,"Toronto, Stn A PO Boxes",43.653482,-79.383935
93,152,M8W,Etobicoke,"Alderwood, Long Branch","Toronto, Alderwood",43.601717,-79.545232
94,153,M9W,Etobicoke,"Northwest, West Humber - Clairville","Toronto, Northwest",43.653482,-79.383935
95,154,M1X,Scarborough,Upper Rouge,"Toronto, Upper Rouge",43.810155,-79.184839
96,157,M4X,Downtown Toronto,"St. James Town, Cabbagetown","Toronto, St. James Town",43.669403,-79.372704
97,158,M5X,Downtown Toronto,"First Canadian Place, Underground city","Toronto, First Canadian Place",43.648768,-79.381692


In [20]:
df1.dropna(subset=['Latitude'],inplace=True)
df1.reset_index()
df1.tail()

,index,PostalCode,Borough,Neighbourhood,address,Latitude,Longitude
98,161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North","Toronto, The Kingsway",43.647381,-79.511333
99,166,M4Y,Downtown Toronto,Church and Wellesley,"Toronto, Church and Wellesley",43.665524,-79.383801
100,169,M7Y,East Toronto,"Business reply mail Processing Centre, South C...","Toronto, Business reply mail Processing Centre",43.653482,-79.383935
101,170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...","Toronto, Old Mill South",43.649826,-79.494334
102,179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...","Toronto, Mimico NW",43.616677,-79.496805


In [21]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

loct= geolocator.geocode('Toronto')
latitude=loct.latitude
longitude=loct.longitude

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto